In [1]:
import os
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime as dt
from datetime import timedelta as td

In [2]:
import utils

In [3]:
RAW_ROOT = 'raw'
DATA_ROOT = 'data'

In [4]:
df_user_raw, df_item_raw, df_log_raw, df_sub_raw = utils.load_raw(RAW_ROOT)

# user data

In [5]:
df_user = df_user_raw.copy()

df_user['FN'].fillna(0, inplace=True)
df_user['Active'].fillna(0, inplace=True)
df_user['club_member_status'].fillna('NONE', inplace=True)
df_user['fashion_news_frequency'].fillna('NONE', inplace=True)
df_user.loc[df_user['fashion_news_frequency'] == 'None', 'fashion_news_frequency'] = 'NONE'

df_user.set_index('customer_id', inplace=True)

df_user = df_user.astype({
    'FN': int,
    'Active': int,
})

In [6]:
if 'postal_code' in df_user.columns:
    df_user = df_user.drop(columns=['postal_code'])

In [7]:
oh_member = pd.get_dummies(df_user['club_member_status']).drop('NONE', axis=1)
oh_newsfreq = pd.get_dummies(df_user['fashion_news_frequency'])

df_user = df_user.drop('club_member_status', axis=1).drop('fashion_news_frequency', axis=1)
df_user = df_user.join(oh_member).join(oh_newsfreq)

In [8]:
df_user.loc[df_user['age'].isna(), 'age'] = round(df_user.loc[df_user['age'].notnull(), 'age'].mean())

from sklearn.mixture import GaussianMixture
gm = GaussianMixture(n_components=2).fit(df_user['age'].values.reshape(-1, 1))
mu1, mu2 = gm.means_.flatten()
var1, var2 = gm.covariances_.flatten()
sigma1, sigma2 = np.sqrt(var1), np.sqrt(var2)
del gm
df_user['age_gmm_1'] = (df_user['age'] - mu1) / sigma1
df_user['age_gmm_2'] = (df_user['age'] - mu2) / sigma2

In [9]:
df_user.head()

,FN,Active,age,ACTIVE,LEFT CLUB,PRE-CREATE,Monthly,NONE,Regularly,age_gmm_1,age_gmm_2
customer_id,,,,,,,,,,,
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,0,0,49.0,1,0,0,0,1,0,-0.035845,4.973913
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,0,0,25.0,1,0,0,0,1,0,-2.415854,-0.068474
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,0,0,24.0,1,0,0,0,1,0,-2.515021,-0.278574
00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c5feb1ca5dff07c43e,0,0,54.0,1,0,0,0,1,0,0.459991,6.024411
00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe7fc0f26dd8d65a85a,1,1,52.0,1,0,0,0,0,1,0.261657,5.604212


In [10]:
df_user.to_parquet(f'{DATA_ROOT}/df_user_preprocessed.pq')

In [11]:
del df_user

# transaction data

In [5]:
df_log = df_log_raw.copy()

dts_to_dto_map = {}
dts_to_ts_map = {}
for dts in df_log_raw['t_dat'].unique():
    dto = dt.strptime(dts, '%Y-%m-%d')
    dts_to_dto_map[dts] = dto
    dts_to_ts_map[dts] = int(dto.timestamp())

df_log['timestamp'] = df_log_raw['t_dat'].map(dts_to_ts_map)
df_log['dto'] = df_log_raw['t_dat'].map(dts_to_dto_map)

df_log['week'] = 104 - (df_log['dto'].max() - df_log['dto']).dt.days // 7

log_price = np.log(df_log['price'])
mu = log_price.mean()
sigma = log_price.std()
df_log['log_price'] = (log_price - mu) / sigma

In [6]:
df_log.head()

,t_dat,customer_id,article_id,price,sales_channel_id,timestamp,dto,week,log_price
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2,1537369200,2018-09-20,0,1.231676
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2,1537369200,2018-09-20,0,0.446524
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2,1537369200,2018-09-20,0,-0.619252
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2,1537369200,2018-09-20,0,-0.457210
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2,1537369200,2018-09-20,0,-0.457210


In [9]:
df_log.to_parquet(f'{DATA_ROOT}/df_log_preprocessed.pq')

In [10]:
del df_log

# item data

In [11]:
df_item = df_item_raw.copy()

df_item.set_index('article_id', inplace=True)

product_group_name_map = {}
for index, name in enumerate(list(df_item['product_group_name'].unique())):
    product_group_name_map[name] = index
product_group_name_map['Unknown'] = -1

df_item['product_group_no'] = df_item['product_group_name'].map(product_group_name_map)

field_pairs = [
    ('product_type_no', 'product_type_name'),
    ('graphical_appearance_no', 'graphical_appearance_name'),
    ('colour_group_code', 'colour_group_name'),
    ('perceived_colour_value_id', 'perceived_colour_value_name'),
    ('perceived_colour_master_id', 'perceived_colour_master_name'),
    ('department_no', 'department_name'),
    ('index_code', 'index_name'),
    ('index_group_no', 'index_group_name'),
    ('section_no', 'section_name'),
    ('garment_group_no', 'garment_group_name'),
]

os.makedirs(f'{DATA_ROOT}/meta', exist_ok=True)
for pair in field_pairs:
    pair_name = '_'.join(pair[0].split('_')[:-1])
    rows = df_item.groupby(list(pair)).size().reset_index().values
    number2name = {}
    for row in rows:
        number, name, _ = row
        number = str(number)
        number2name[number] = name
    with open(f'{DATA_ROOT}/meta/map_{pair_name}.pkl', 'wb') as fp:
        pickle.dump(number2name, fp)

fields_with_unknown = [
    'product_type_no',
    'graphical_appearance_no',
    'colour_group_code',
    'perceived_colour_value_id',
    'perceived_colour_master_id',
    'garment_group_no',
]

df_item.loc[df_item['garment_group_no'] == 1001, 'garment_group_no'] = -1

In [12]:
iids = df_item.index.to_list()
iids_with_no_desc = df_item[df_item['detail_desc'].isna()].index.to_list()

ss_item_count = df_log_raw.groupby('article_id')['customer_id'].count()
ss_item_count.name = 'transaction_count'

df_item_yes_desc = df_item[~df_item['detail_desc'].isna()]
df_item_yes_desc = df_item_yes_desc.merge(ss_item_count, how='left', left_index=True, right_index=True)
df_item_yes_desc['transaction_count'].fillna(0, inplace=True)

most_popular_desc = df_item_yes_desc.sort_values('transaction_count').iloc[-1].detail_desc

similar_condition_fields = [
    'garment_group_no',
    'product_group_no',
    'perceived_colour_master_id',
    'product_type_no',
    'section_no',
    'index_code',
    'perceived_colour_value_id',
    'colour_group_code',
    'graphical_appearance_no',
    'department_no',
    'product_code',
]

trials = []
for iid in iids_with_no_desc:
    row = df_item.loc[iid]
    desc = most_popular_desc
    for i in range(len(similar_condition_fields) - 1):
        cut = len(similar_condition_fields) - i
        fields = [field for field in similar_condition_fields[:cut] if row[field] != -1]
        cond = (df_item_yes_desc[fields[0]] == row[fields[0]])
        for j in range(1, len(fields)):
            field = fields[j]
            cond = cond & (df_item_yes_desc[field] == row[field])
        df_item_candidates = df_item_yes_desc[cond]
        count = len(df_item_candidates)
        if count:
            desc = df_item_candidates.sort_values('transaction_count').iloc[-1]['detail_desc']
            trials.append(i)
            break
    df_item.loc[iid, 'detail_desc'] = desc
trials = np.array(trials)
print(f"평균 {trials.mean():.02f}개의 조건을 제거했어야했음 (표준편차: {trials.std():.02f})")

평균 1.68개의 조건을 제거했어야했음 (표준편차: 1.32)


In [13]:
df_item.drop(columns=[
    'product_type_name',
    'product_group_name',
    'graphical_appearance_name',
    'colour_group_name',
    'perceived_colour_value_name',
    'perceived_colour_master_name',
    'department_name',
    'index_name',
    'index_group_name',
    'section_name',
    'garment_group_name',
], inplace=True)

In [14]:
df_item.to_parquet(f'{DATA_ROOT}/df_item_preprocessed.pq')

In [15]:
del df_item

# aggregation

In [30]:
df_user = pd.read_parquet(f'{DATA_ROOT}/df_user_preprocessed.pq')
df_item = pd.read_parquet(f'{DATA_ROOT}/df_item_preprocessed.pq')
df_log_all = pd.read_parquet(f'{DATA_ROOT}/df_log_preprocessed.pq')

In [31]:
DNAME = 'CV'

df_log = utils.get_df_log_of(df_log_all, DNAME)
df_log = df_log[df_log['target'] != 'test']

In [32]:
del df_log_all

In [33]:
df_channel_count = df_log['sales_channel_id'].value_counts()
global_online_prob = df_channel_count.loc[2] / (df_channel_count.loc[2] + df_channel_count.loc[1])

ss_sc_online_count = df_log[df_log['sales_channel_id'] == 2].groupby('customer_id').size()
ss_sc_offline_count = df_log[df_log['sales_channel_id'] == 1].groupby('customer_id').size()

df_user['online_prob'] = ss_sc_online_count / (ss_sc_online_count + ss_sc_offline_count)

del ss_sc_online_count
del ss_sc_offline_count

df_user['online_prob'].fillna(global_online_prob, inplace=True)

In [34]:
df_item_with_cohort_group = df_item.copy()

df_item_with_cohort_group['cohort_group'] = df_item_with_cohort_group['index_code'].copy()

df_log_with_cohort_group = df_log.merge(df_item_with_cohort_group['cohort_group'], how='left', left_on='article_id', right_index=True)

del df_item_with_cohort_group

cohort_group_columns = ['A', 'B', 'C', 'D', 'F', 'G', 'H', 'I', 'J', 'S']
df_user_of_cohort_group_counts = df_log_with_cohort_group[['customer_id', 'article_id', 'cohort_group']].groupby(['customer_id', 'cohort_group']).count().unstack()
df_user_of_cohort_group_counts.fillna(0, inplace=True)
df_user_of_cohort_group_counts.columns = cohort_group_columns

df_cohort_count = df_log_with_cohort_group['cohort_group'].value_counts()
df_cohort_dist = df_cohort_count.copy()
df_cohort_dist = df_cohort_dist / df_cohort_dist.sum()

del df_log_with_cohort_group

df_user_of_cohort_group_dist = df_user_of_cohort_group_counts.div(df_user_of_cohort_group_counts.sum(1), axis='index')
df_user_of_cohort_group_dist.columns = [f'cohort_prob_{code}' for code in cohort_group_columns]
df_user = df_user.merge(df_user_of_cohort_group_dist, how='left', left_index=True, right_index=True)

del df_user_of_cohort_group_dist
del df_user_of_cohort_group_counts

for code in cohort_group_columns:
    df_user[f'cohort_prob_{code}'].fillna(df_cohort_dist.loc[code], inplace=True)

In [35]:
df_item_with_garment_group = df_item.copy()

df_item_with_garment_group['garment_group'] = df_item_with_garment_group['garment_group_no'].copy()
df_item_with_garment_group = df_item_with_garment_group[df_item_with_garment_group['garment_group'] != -1]

df_log_with_garment_group = df_log.merge(df_item_with_garment_group['garment_group'], how='inner', left_on='article_id', right_index=True)
df_log_with_garment_group = df_log_with_garment_group.astype({
    'garment_group': int,
})

del df_item_with_garment_group

df_garment_count = df_log_with_garment_group['garment_group'].value_counts()
df_garment_dist = df_garment_count.copy()
df_garment_dist = df_garment_dist / df_garment_dist.sum()

df_user_of_garment_group_counts = df_log_with_garment_group[['customer_id', 'article_id', 'garment_group']].groupby(['customer_id', 'garment_group']).count().unstack()
df_user_of_garment_group_counts.fillna(0, inplace=True)
df_user_of_garment_group_counts.columns = [f'garment_prob_{garment_group}' for _, garment_group in df_user_of_garment_group_counts.columns.to_list()]

del df_log_with_garment_group

df_user_of_garment_group_dist = df_user_of_garment_group_counts.div(df_user_of_garment_group_counts.sum(1), axis='index')
df_user = df_user.merge(df_user_of_garment_group_dist, how='left', left_index=True, right_index=True)

del df_user_of_garment_group_dist
del df_user_of_garment_group_counts

for index in df_garment_dist.index.to_list():
    df_user[f'garment_prob_{index}'].fillna(df_garment_dist.loc[index], inplace=True)

In [36]:
df_user_of_avg_price = df_log.groupby('customer_id')['price'].mean('price')
df_user_of_avg_price.name = 'avg_price'

global_avg_price = df_log['price'].mean()

df_user = df_user.merge(df_user_of_avg_price, how='left', left_index=True, right_index=True)
df_user['avg_price'].fillna(global_avg_price, inplace=True)

In [37]:
ss_transaction_count = df_log.groupby('customer_id').size()
ss_transaction_count.name = 'transaction_count'

df_user = df_user.merge(ss_transaction_count, how='left', left_index=True, right_index=True)
df_user['transaction_count'].fillna(0, inplace=True)

del ss_transaction_count

cold_threshold = int(np.percentile(df_user['transaction_count'], 50))
df_user['is_cold'] = 0
df_user.loc[df_user['transaction_count'] < cold_threshold, 'is_cold'] = 1

df_user['is_new'] = 0
df_user.loc[df_user['transaction_count'] < 1, 'is_new'] = 1

df_user['log_transaction_count'] = np.log(1 + df_user['transaction_count'])

In [38]:
df_user_of_week = df_log.groupby('customer_id').aggregate({'week': 'max'})
df_user_of_week.columns = ['last_active_week']

df_user = df_user.merge(df_user_of_week, how='left', left_index=True, right_index=True)
df_user['last_active_week'].fillna(0, inplace=True)
df_user = df_user.astype({
    'last_active_week': int,
})

In [39]:
df_user.to_parquet(f'{DATA_ROOT}/df_user_aggregated_{DNAME}.pq')

In [40]:
del df_user
del df_log

# aggregate item

In [50]:
df_item = pd.read_parquet(f'{DATA_ROOT}/df_item_preprocessed.pq')
df_log_all = pd.read_parquet(f'{DATA_ROOT}/df_log_preprocessed.pq')

In [51]:
DNAME = 'CV'

df_log = utils.get_df_log_of(df_log_all, DNAME)
df_log = df_log[df_log['target'] != 'test']

In [52]:
del df_log_all

In [53]:
df_channel_count = df_log['sales_channel_id'].value_counts()

global_online_prob = df_channel_count.loc[2] / (df_channel_count.loc[2] + df_channel_count.loc[1])

ss_sc_online_count = df_log[df_log['sales_channel_id'] == 2].groupby('article_id').size()
ss_sc_offline_count = df_log[df_log['sales_channel_id'] == 1].groupby('article_id').size()

df_item['online_prob'] = ss_sc_online_count / (ss_sc_online_count + ss_sc_offline_count)

del ss_sc_online_count
del ss_sc_offline_count

df_item['online_prob'].fillna(global_online_prob, inplace=True)

In [54]:
df_item_of_avg_price = df_log.groupby('article_id')['price'].mean('price')
df_item_of_avg_price.name = 'avg_price'

global_avg_price = df_log['price'].mean()

df_item = df_item.merge(df_item_of_avg_price, how='left', left_index=True, right_index=True)
df_item['avg_price'].fillna(global_avg_price, inplace=True)

In [55]:
month2season = {
    1: 'winter',
    2: 'winter',
    3: 'spring',
    4: 'spring',
    5: 'spring',
    6: 'summer',
    7: 'summer',
    8: 'summer',
    9: 'fall',
    10: 'fall',
    11: 'fall',
    12: 'winter',
}

dts_to_season_map = {}
for dts in df_log['t_dat'].unique():
    month = int(dts.split('-')[1])
    season = month2season[month]
    dts_to_season_map[dts] = season

df_log['season'] = df_log['t_dat'].map(dts_to_season_map)

season_to_count_map = df_log['season'].value_counts().sort_values(ascending=False).to_dict()

df_season_count = df_log['season'].value_counts()
df_season_dist = df_season_count.copy()
df_season_dist = df_season_dist / df_season_dist.sum()

df_item_of_season_counts = df_log[['customer_id', 'article_id', 'season']].groupby(['article_id', 'season']).count().unstack()
df_item_of_season_counts.fillna(0, inplace=True)
df_item_of_season_counts.columns = [f'season_prob_{season}' for _, season in df_item_of_season_counts.columns.to_list()]

df_item_of_season_dist = df_item_of_season_counts.div(df_item_of_season_counts.sum(1), axis='index')
df_item = df_item.merge(df_item_of_season_dist, how='left', left_index=True, right_index=True)

del df_item_of_season_counts
del df_item_of_season_dist

for index in df_season_dist.index.to_list():
    df_item[f'season_prob_{index}'].fillna(df_season_dist.loc[index], inplace=True)

In [56]:
ss_transaction_count = df_log.groupby('article_id').size()
ss_transaction_count.name = 'transaction_count'

df_item = df_item.merge(ss_transaction_count, how='left', left_index=True, right_index=True)
df_item['transaction_count'].fillna(0, inplace=True)

del ss_transaction_count

cold_threshold = int(np.percentile(df_item['transaction_count'], 50))
df_item['is_cold'] = 0
df_item.loc[df_item['transaction_count'] < cold_threshold, 'is_cold'] = 1

df_item['is_new'] = 0
df_item.loc[df_item['transaction_count'] < 1, 'is_new'] = 1

df_item['log_transaction_count'] = np.log(1 + df_item['transaction_count'])

In [57]:
df_item.to_parquet(f'{DATA_ROOT}/df_item_aggregated_{DNAME}.pq')

In [58]:
del df_item
del df_log

In [59]:
del df_user_raw
del df_item_raw
del df_log_raw
del df_sub_raw

# feature

In [12]:
DNAME = 'CV'

In [13]:
df_user = pd.read_parquet(f'{DATA_ROOT}/df_user_aggregated_{DNAME}.pq')

df_user_feature = df_user.copy()
del df_user

df_user_feature.drop(columns=[
    'age',
    'transaction_count',
], inplace=True)

one_hot_fields = [
    'FN',
    'Active',
    'is_cold',
]
for field in one_hot_fields:
    hotted = pd.get_dummies(df_user_feature[field], prefix=field)
    df_user_feature = df_user_feature.merge(hotted, how='left', left_index=True, right_index=True)
    df_user_feature.drop(columns=[field], inplace=True)

df_user_feature.to_parquet(f'{DATA_ROOT}/df_user_feature_{DNAME}.pq')

In [14]:
uid2law = {}
for uid, law in df_user_feature[['last_active_week']].reset_index().values:
    uid2law[uid] = law
df_user_feature.drop(columns=['last_active_week'], inplace=True)

with open(f'{DATA_ROOT}/uid2law_{DNAME}.pkl', 'wb') as fp:
    pickle.dump(uid2law, fp)

del uid2law

In [15]:
uid2feature = {}
for row in df_user_feature.reset_index().values:
    uid = row[0]
    feature = tuple(row[1:])
    uid2feature[uid] = feature

with open(f'{DATA_ROOT}/uid2feature_{DNAME}.pkl', 'wb') as fp:
    pickle.dump(uid2feature, fp)

del uid2feature

# item feature

In [22]:
DNAME = 'LB'

In [23]:
df_item = pd.read_parquet(f'{DATA_ROOT}/df_item_aggregated_{DNAME}.pq')

df_item_feature = df_item.copy()

iid2pid = {}
iid2title = {}
# iid2ipath = {}
iid2desc = {}

for row in df_item_feature[['product_code', 'prod_name', 'detail_desc']].itertuples():
    iid, pid, title, desc = row
    iid2pid[iid] = pid
    iid2title[iid] = title
    # iid2ipath[iid] = ipath
    iid2desc[iid] = desc

with open(f'{DATA_ROOT}/iid2pid.pkl', 'wb') as fp:
    pickle.dump(iid2pid, fp)
with open(f'{DATA_ROOT}/iid2title.pkl', 'wb') as fp:
    pickle.dump(iid2title, fp)
# with open(f'{DATA_ROOT}/iid2ipath.pkl', 'wb') as fp:
#     pickle.dump(iid2ipath, fp)
with open(f'{DATA_ROOT}/iid2desc.pkl', 'wb') as fp:
    pickle.dump(iid2desc, fp)

del iid2pid
del iid2title
# del iid2ipath
del iid2desc

df_item_feature.drop(columns=[
    'product_code',
    'prod_name',
    'detail_desc',
    'transaction_count',
], inplace=True)

one_hot_fields = [
    'product_type_no',
    'graphical_appearance_no',
    'colour_group_code',
    'perceived_colour_value_id',
    'perceived_colour_master_id',
    'department_no',
    'index_code',
    'section_no',
    'garment_group_no',
    'product_group_no',
]

fields_with_unknown = [
    'product_group_no',
    'product_type_no',
    'graphical_appearance_no',
    'colour_group_code',
    'perceived_colour_value_id',
    'perceived_colour_master_id',
    'garment_group_no',
]

for field in one_hot_fields:
    hotted = pd.get_dummies(df_item_feature[field], prefix=field)
    if field in fields_with_unknown:
        hotted = pd.get_dummies(df_item_feature[field], prefix=field)
        unknown_field_name = f'{field}_-1'
        unknowns_rows = (hotted[unknown_field_name] == 1)
        hotted.loc[unknowns_rows, :] = 1 / (len(hotted.columns) - 1)
        hotted.drop(columns=[unknown_field_name], inplace=True)
    df_item_feature = df_item_feature.merge(hotted, how='left', left_index=True, right_index=True)
    df_item_feature.drop(columns=[field], inplace=True)

df_item_feature.to_parquet(f'{DATA_ROOT}/df_item_feature_{DNAME}.pq')

In [24]:
iid2feature = {}
for row in df_item_feature.reset_index().values:
    iid = row[0]
    feature = tuple(row[1:])
    iid2feature[iid] = feature

del df_item_feature

with open(f'{DATA_ROOT}/iid2feature_{DNAME}.pkl', 'wb') as fp:
    pickle.dump(iid2feature, fp)

del iid2feature